# Prepare to start <a class="anchor" id="prepare"></a>

## Loading packages

In [1]:
import os
import pydicom as dicom
import pandas as pd
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")

# Select Images to train our model

In [3]:
segmentations_path = "../input/rsna-2022-cervical-spine-fracture-detection/segmentations/"
patients_to_train = os.listdir(segmentations_path)
df = pd.DataFrame()

for i in range (0, len(patients_to_train)):
    

    segmentations = nib.load(segmentations_path + patients_to_train[i]).get_fdata()[:, ::-1, ::-1].transpose(2, 1, 0)
    study = patients_to_train[i][:-4] #removing extention
    
    for j in range (0, segmentations.shape[0]):
        if (np.unique(segmentations[j]).size == 2): #returns the IMAGES that only shows one vertebrae in the axial pane. (note that some images contains multiple bones and that can be confusing for the algorithm)
              
            label = np.unique(segmentations[j])[1] #detecting label. ex: C1.
            if (label == 1): # after detecting label, checking if it's fractured or not. '0' NOT FRACTURED - '1' FRACTURED. "values[0] contains this feature"
                fractured = train[train.StudyInstanceUID == study]["C1"].values[0]
            elif (label == 2):
                fractured = train[train.StudyInstanceUID == study]["C2"].values[0]
            elif (label == 3):
                fractured = train[train.StudyInstanceUID == study]["C3"].values[0]
            elif (label == 4):
                fractured = train[train.StudyInstanceUID == study]["C4"].values[0]
            elif (label == 5):
                fractured = train[train.StudyInstanceUID == study]["C5"].values[0]
            elif (label == 6):
                fractured = train[train.StudyInstanceUID == study]["C6"].values[0]
            elif (label == 7):
                fractured = train[train.StudyInstanceUID == study]["C7"].values[0]
          

            new_row = {'StudyInstanceUID': study, 'Image': j, 'Label': label, 'Fractured': fractured}
            df = df.append(new_row, ignore_index=True)


In [4]:
df_final = df[df["Label"]<8]
df_final.reset_index(inplace=True)
df_final

,index,StudyInstanceUID,Image,Label,Fractured
0,0,1.2.826.0.1.3680043.780,60.0,1.0,0.0
1,1,1.2.826.0.1.3680043.780,61.0,1.0,0.0
2,2,1.2.826.0.1.3680043.780,62.0,1.0,0.0
3,3,1.2.826.0.1.3680043.780,84.0,2.0,1.0
4,4,1.2.826.0.1.3680043.780,85.0,2.0,1.0
...,...,...,...,...,...
3781,4915,1.2.826.0.1.3680043.31077,373.0,7.0,0.0
3782,4916,1.2.826.0.1.3680043.31077,374.0,7.0,0.0
3783,4917,1.2.826.0.1.3680043.31077,375.0,7.0,0.0
3784,4918,1.2.826.0.1.3680043.31077,376.0,7.0,0.0


In [5]:
df_final.to_csv("train_filtered.csv")